# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка
Прежде чем рабодать с данными, импортируем необходимые инструменты.

In [ ]:
import os.path
import pandas as pd
import numpy as np
import re
import os
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV
from nltk.corpus import stopwords
import nltk
from sklearn.pipeline import Pipeline
from tqdm.notebook import tqdm

Используем собственную функцию для загрузки данных.

In [ ]:
def load_data(path, col_sep=",", float_sep=".", data=None):
    """Функция для загрузки данных из CSV-файла локально или удаленно."""
    for instance in path, f"/{path}":
        if os.path.exists(instance):
            data = pd.read_csv(
                instance, sep=col_sep, decimal=float_sep, engine="python"
            )
            break

    if data is None:
        try:
            data = pd.read_csv(
                f"https://code.s3.yandex.net/{path}",
                sep=col_sep,
                decimal=float_sep,
                engine="python",
            )
        except IOError:
            print("Файл с данными не найден.")

    return data

In [ ]:
# Загрузка данных
df = load_data("datasets/toxic_comments.csv")

if df is not None:
    df.info()
    display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
# Анализ баланса классов
print("Распределение классов в данных:")
print(df['toxic'].value_counts())

Распределение классов в данных:
0    143106
1     16186
Name: toxic, dtype: int64


На текущем этапе классы не сбалансированы: 0 (нетоксичные) значительно преобладают над 1 (токсичные). Однако в дальнейшем кмы учтем это н аэтапе парметров модели для обучения.

In [ ]:
# Предобработка текста

nltk.download('stopwords')
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

stop_words = set(stopwords.words("english"))

tqdm.pandas()

def preprocess_texts(texts):
    cleaned_texts = [re.sub(r'[^a-z ]', ' ', text.lower()).strip() for text in texts]
    lemmatized_texts = []
    for doc in tqdm(nlp.pipe(cleaned_texts, disable=["parser", "ner"]), total=len(cleaned_texts)):
        lemmatized_texts.append(' '.join([token.lemma_ for token in doc if token.text not in stop_words]))
    return lemmatized_texts

df['text'] = preprocess_texts(df['text'].values)


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  0%|          | 0/159292 [00:00<?, ?it/s]

In [ ]:
if df is not None:
    df.info()
    display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


,Unnamed: 0,text,toxic
0,0,explanation edit make username hardcore metall...,0
1,1,aww match background colour seemingly stuck ...,0
2,2,hey man really try edit war guy constantly...,0
3,3,make real suggestion improvement wonder sec...,0
4,4,sir hero chance remember page,0


In [ ]:
# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['toxic'], test_size=0.2, random_state=42, stratify=df['toxic']
)

## Обучение

In [ ]:
# Создание пайплайнов с TF-IDF и моделями
pipeline_lr = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=50000, ngram_range=(1, 2))),
    ('model', LogisticRegression(solver='lbfgs', max_iter=500, random_state=42))
])

pipeline_sgd = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=50000, ngram_range=(1, 2))),
    ('model', SGDClassifier(loss='hinge', class_weight='balanced', random_state=42))
])


In [ ]:
# Поиск гиперпараметров через RandomizedSearchCV для регресии
param_dist_lr = {'model__C': np.logspace(-3, 2, 6)}
search_lr = RandomizedSearchCV(pipeline_lr, param_dist_lr, scoring='f1', cv=3, n_iter=5, random_state=42)
search_lr.fit(X_train, y_train)


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('tfidf',
                                              TfidfVectorizer(max_features=50000,
                                                              ngram_range=(1,
                                                                           2))),
                                             ('model',
                                              LogisticRegression(max_iter=500,
                                                                 random_state=42))]),
                   n_iter=5,
                   param_distributions={'model__C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
                   random_state=42, scoring='f1')

In [ ]:
# Поиск гиперпараметров через RandomizedSearchCV для SGD
param_dist_sgd = {'model__alpha': np.logspace(-4, 0, 5)}
search_sgd = RandomizedSearchCV(pipeline_sgd, param_dist_sgd, scoring='f1', cv=3, n_iter=5, random_state=42)
search_sgd.fit(X_train, y_train)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('tfidf',
                                              TfidfVectorizer(max_features=50000,
                                                              ngram_range=(1,
                                                                           2))),
                                             ('model',
                                              SGDClassifier(class_weight='balanced',
                                                            random_state=42))]),
                   n_iter=5,
                   param_distributions={'model__alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00])},
                   random_state=42, scoring='f1')

In [ ]:
# Выбор лучшей модели по кросс-валидации
best_model = search_lr if search_lr.best_score_ > search_sgd.best_score_ else search_sgd
print(f"Лучшая модель: {'Logistic Regression' if best_model == search_lr else 'SGDClassifier'}")


Лучшая модель: Logistic Regression


In [ ]:
# Обучение лучшей модели на всех тренировочных данных
best_model.best_estimator_.fit(X_train, y_train)

# Оценка на тестовой выборке
y_pred_best = best_model.best_estimator_.predict(X_test)
f1_best = f1_score(y_test, y_pred_best)
print(f"F1-score (Best Model): {f1_best:.4f}")

F1-score (Best Model): 0.7807


Для преобразования текстов в числовое представление использовался метод `TF-IDF` с учетом униграмм и биграмм. Чтобы избежать утечки данных при кросс-валидации, была использована конструкция `Pipeline`, включающая векторизацию и модель. В качестве моделей были обучены логистическая регрессия и линейный `SGD`-классификатор. Для каждой из них подобраны оптимальные гиперпараметры с помощью `RandomizedSearchCV`.

## Выводы

Благодаря внесенным улучшениям удалось достичь надежных значений `F1-score`. По результатам кросс-валидации была выбрана лучшая модель — логистическая регрессия, которая показала `F1-score` **0.7807** на тестовой выборке. Такой результат свидетельствует о высокой эффективности подхода к классификации токсичных комментариев. Правильная стратегия выбора модели позволила избежать утечки данных и сделать оценку более надежной.